In [17]:
from pnlp import piop
from pnlp import ptxt
from pnlp import pmag
import os
import math
import re
import json
import time
import random

from collections import Counter
from aip import AipNlp

import datetime
from bs4 import BeautifulSoup
import requests
import shutil

from pypinyin import pinyin, lazy_pinyin, Style

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
BASE_URL = "https://www.lagou.com/zhaopin/"

In [3]:
cate_gw = piop.read_yml('./data/category/category.yml')

In [4]:
cate_url = piop.read_yml('./data/category/category_url_jointag.yml')

In [5]:
urlist_path = "./data/position_urlist/"
html_path = "./data/html/"
out_path = ("./data/extract/")
seg_path = "./data/segpos/"
model_path = "./model/"

## 分类预处理

In [108]:
res = []
for key, value in cate_gw.items():
    for key2, value2 in value.items():
        tmp = []
        for item in value2.split(";"):
            pylist = lazy_pinyin(item)
            url = BASE_URL + "".join(pylist) + "/"
            tmp.append("".join(pylist))
            #print(key2, url)
        res.append(";".join(tmp))

In [36]:
resurl = []
n = 0
for k,v in cate_url.items():
    for k2, v2 in v.items():
        lenth = (len(v2.split(";")))
        resurl.append((k2,lenth))
        n += lenth
n

302

## 爬虫

### 按类别获取所有职位链接

In [7]:
def get_urlist(title):
    res = []
    gangwei_base_url = BASE_URL + title + "/"
#     arr = [w for w in range(1, 31)]
    for i in range(1, 31):
#         choose = random.randint(0, len(arr)-1)
#         i = arr.pop(choose)
        url = gangwei_base_url + str(i) + "/?filterOption=2"
        page_urlist = get_pageurlist(url)
        if len(page_urlist) > 0:
            res.extend(page_urlist)
        else:
            break
        sec = random.randint(1, 3)
        time.sleep(sec)
        
        # 每 10 个重启一次
        if i % 10 == 0:
            driver.close()
            driver = webdriver.Chrome()
            driver.maximize_window()
    return res

In [8]:
def get_pageurlist(url):
    position_links = []
    driver.get(url)
    driver.execute_script("window.scrollBy(0,100)")
    ps = driver.page_source
    soup = BeautifulSoup(ps)
    is404 = soup.find("p", {"class": "tip"})
    if is404:
        print(url, is404.get_text())
    else:
        position_links = soup.find_all("a", {"class": "position_link"})
        position_links = [pl.get("href") for pl in position_links]
    return position_links

In [105]:
# flat all categories
all_items = []
for cate, gangwei_item in cate_gw.items():
    for gangwei, zhiwei in gangwei_item.items():
        zhiwei_list = zhiwei.split(";")
        zhiwei_title_list = cate_url[cate][gangwei].split(";")
        for i, zw in enumerate(zhiwei_list):
            all_items.append((cate, gangwei, zw, zhiwei_title_list[i]))

In [191]:
driver = webdriver.Chrome()
driver.maximize_window()

In [599]:
for item in all_items:
    
    url_dict = {}

    cate = item[0]
    gangwei = item[1]
    zw = item[2]
    title = item[3]

    urlist = get_urlist(title)
    
    # 不满 30 页的
    if len(urlist) < 15*30:
        print(cate, "\t", gangwei, "\t", zw, "\t", len(urlist))

    url_dict['position'] = cate + "_" + gangwei + "_" + zw + "_" + title
    url_dict['urlist'] = urlist

    out_fpath = os.path.join(urlist_path, url_dict['position'] + ".txt")
    iopipe.write_json(out_fpath, url_dict, indent=4, ensure_ascii=False)

### 按类别下载所有职位页面

In [507]:
def write_html(fpath, page_source):
    with open(fpath, 'w') as f:
        f.write(page_source)

In [508]:
def download():
    driver = webdriver.Chrome()
    filelist = os.listdir(urlist_path)
    for file in filelist:
        if file[0] == '.':
            continue
        data = piop.read_json(urlist_path + file)
        fpath = html_path + data['position']
        urlist = data['urlist']
        for i, url in enumerate(urlist):
            fname = os.path.split(url)[-1]
            if os.path.exists(fpath + "/" + fname):
                continue

            driver.get(url)
            ps = driver.page_source
            write_html(fpath + "/" + fname, ps)
            time.sleep(random.randint(1, 3))

            # 每 10 个重启一次
            if i % 10 == 0:
                driver.close()
                driver = webdriver.Chrome()

In [69]:
# 重复 url 检验
filelist = os.listdir(urlist_path)
for file in filelist:
    if file[0] == '.':
        continue
    data = piop.read_json(urlist_path + file)
    fpath = html_path + data['position']
    urlist = data['urlist']
    filenum = len([_ for _ in os.listdir(fpath) if _[0] != '.'])
    urlnum = len(urlist)
    if filenum != urlnum:
        print(file, filenum, urlnum)
    if filenum != len(set(urlist)):
        print("wrong")

技术_后端开发_区块链_qukuailian.txt 430 435
技术_企业软件_售前工程师_shouqiangongchengshi.txt 423 435
职能_行政_助理_zhuli.txt 434 435
技术_运维_运维开发工程师_yunweikaifagongchengshi.txt 420 435
设计_视觉设计_原画师_yuanhuashi.txt 421 435
技术_后端开发_Go_go.txt 417 435
技术_运维_系统安全_xitonganquan.txt 425 435
游戏_其他_电竞主持_dianjingzhuchi.txt 428 435
技术_后端开发_Python_Python.txt 433 435
运营_运营_运营专员_yunyingzhuanyuan.txt 433 435
产品_产品设计师_无线产品设计师_wuxianchanpinshejishi.txt 92 99
产品_产品经理_产品实习生_chanpinshixisheng.txt 413 424
技术_后端开发_全栈工程师_quanzhangongchengshi.txt 422 435
运营_运营_活动运营_huodongyunying.txt 434 435
技术_硬件开发_电路设计_dianlusheji.txt 434 435
产品_产品经理_产品经理_chanpinjingli.txt 433 435
运营_运营_运营经理_yunyingjingli.txt 434 435
职能_财务_财务_caiwu.txt 433 435
技术_后端开发_C++_C++.txt 430 435
技术_高端职位_高端技术职位其它_gaoduanjishuzhiweiqita.txt 431 435
运营_运营_产品运营_chanpinyunying.txt 389 435
产品_产品经理_数据产品经理_shujuchanpinjingli.txt 427 435
设计_用户研究_数据分析师_shujufenxishi.txt 426 435
产品_产品经理_产品助理_chanpinzhuli.txt 428 435


In [220]:
# 将重复的下载页面用正确完整的页面替代
for pname in os.listdir(html_path):
    if pname[0] == '.':
        continue
    for base_file in os.listdir(html_path+pname):
        tmp1, tmp2 = [], []
        for pname in os.listdir(html_path):
            if pname[0] == '.':
                continue
            for file in os.listdir(html_path+pname):
                if file == base_file:
                    base = os.path.join(html_path, pname, file)
                    size = os.path.getsize(base)
                    tmp1.append(base)
                    tmp2.append(size)
        if len(set(tmp2)) == 1:
            continue
        maxindex = tmp2.index(max(tmp2))
        maxfile = tmp1[maxindex]
        tmp1.remove(maxfile)
        for ofile in tmp1:
#             if os.path.getsize(ofile) < 15000:
            shutil.copyfile(maxfile, ofile)
#         print(base_file, "done")

In [ ]:
# 处理 404 页面
urlset = []
n = 0
cal = 0
driver = webdriver.Chrome()
filelist = os.listdir(urlist_path)
for file in filelist:
    if file[0] == '.':
        continue
    data = piop.read_json(urlist_path + file)
    fpath = html_path + data['position']
    urlist = list(set(data['urlist']))
    for i, url in enumerate(urlist):
        fname = os.path.split(url)[-1]
#         if os.path.exists(fpath + "/" + fname):
#             n += 1
#             urlset.append(url)
#             continue
#         downps = piop.read_file(fpath + "/" + fname)
#         soup = BeautifulSoup(downps)
#         is404 = soup.find("h4", {"class": "error_msg"})
        size = os.path.getsize(fpath + "/" + fname)
        if size == 207786:
            cal += 1
#             print(fpath + "/" + fname)
#             print(url)
#             print(url, is404.get_text())

            driver.get(url)
            ps = driver.page_source
            write_html(fpath + "/" + fname, ps)
            time.sleep(random.randint(1, 3))

            # 每 10 个重启一次
            if cal % 10 == 0:
                driver.close()
                driver = webdriver.Chrome()

In [250]:
# 57905 个不重复页面
# 共 92842 个页面

In [ ]:
p = Pool()
for i in range(2):
    p.apply_async(download, args=(i, ))
p.close()
p.join()

### 按类别解析所有职位页面

#### 预处理

In [613]:
# 页面检查
allist = []
filelist = os.listdir(urlist_path)
for file in filelist:
    if file[0] == '.':
        continue
    data = piop.read_json(urlist_path + file)
    fpath = html_path + data['position']
    urlist = list(set(data['urlist']))
    allist.extend(urlist)
print(len(allist), len(set(allist)))

allfile = []
for pname in os.listdir(html_path):
    if pname[0] == '.':
        continue
    for base_file in os.listdir(html_path+pname):
        allfile.append(base_file)
print(len(allfile), len(set(allfile)))

92842 57905
92842 57905


In [593]:
# 输出职位职责和职位要求的说明
res = dict()
tmp = dict()
for pname in os.listdir(html_path):
    if pname[0] == '.':
        continue
    for base_file in os.listdir(html_path+pname):
        if base_file in tmp:
            continue
        tmp[base_file] = ""
        html_file = os.path.join(html_path, pname, base_file)
        raw_html = piop.read_file(html_file)
        soup = BeautifulSoup(raw_html)
        try:
            jobdetail = soup.find("div", {"class": "job-detail"}).get_text()
        except Exception as e:
            continue
        for item in re.compile(r"\n+").split(jobdetail):
            if re.compile(r'[\da-zA-Z]+').search(item) or len(item) == 0 or len(item) > 8:
                continue
            item = "||".join(ptxt.Text(item, 'chi').extract.mats)
            if item not in res:
                res[item] = 1
            else:
                res[item] += 1

In [597]:
len(res)

4781

In [598]:
sorted_res = sorted(res.items(), key=lambda x:x[1], reverse=True)

In [606]:
with open("/Users/HaoShaochun/Desktop/job_description_mark_sorted_dict.txt", 'w') as f:
    for item in sorted_res:
        item = item[0] + "||" + str(item[1])
        f.write(item+"\n")

#### 解析

In [1043]:
DTIMEDICT = {
    '1天前': "2019-05-11",
    "2天前": "2019-05-10", 
    "3天前": "2019-05-09"
}

In [1042]:
JOB_DESC_MARK_DICT = piop.read_lines(
    "/Users/HaoShaochun/Desktop/job_description_mark_sorted_dict.txt")

In [ ]:
for pname in os.listdir(html_path):
    if pname[0] == '.':
        continue
    print("="*20, pname, "BEGIN", "="*20)
    out_file = os.path.join(out_path, pname + ".txt")
    if os.path.exists(out_file):
        continue
    res = []
    for base_file in os.listdir(html_path+pname):
        html_file = os.path.join(html_path, pname, base_file)
        try:
            item = get_item(html_file)
            res.append(item)
        except Exception as e:
            print(html_file, e)
            continue
    piop.write_json(out_file, res, indent=4, ensure_ascii=False)
    print("="*20, pname, "DONE", "="*20 + "\n\n")
#     break

In [736]:
def split_duty_require(jobdetail):
    res = {}
    tmp = []
    for item in re.compile(r'\n+').split(jobdetail):
        item_chi = "".join(ptxt.Text(item, 'chi').extract.mats)
        if len(item_chi) == 0 or len(item_chi) > 8:
            continue
        for mark in JOB_DESC_MARK_DICT:
            if mark in item:
                tmp.append(mark)
    tmp = sorted(tmp, key=lambda x: len(x), reverse=True)[:2]
    rex  = "|".join(tmp)
    reg = re.compile(rf'{rex}')
    reslist = reg.split(jobdetail)
    reslist = [_ for _ in reslist if len(ptxt.Text(_, 'chi').extract.mats) > 1]
    return reslist

In [1077]:
def get_item(html_file):
    
    item = {}
    raw_html = piop.read_file(html_file)
    soup = BeautifulSoup(raw_html)

    content = soup.find("div", {"class": "position-content-l"})


    company = content.find("div", {"class": "company"}).get_text()
    jobname = content.find("span", {"class": "name"}).get_text()
    salary = content.find("span", {"class": "salary"}).get_text()
    labels = content.find("ul", {"class": "position-label clearfix"}
                         ).get_text().replace("\n", ",")[1:-1]
    request = content.find("p").get_text().split("\n/")[-1].replace(" /\n", ",")[:-1]
    dtime = content.find("p", {"class": "publish_time"}).get_text().split("\xa0")[0]
    if ":" in dtime:
        dtime = "2019-05-12"
    elif dtime in DTIMEDICT:
        dtime = DTIMEDICT[dtime]
    else:
        dtime = dtime

    jobadv = soup.find("dd", {"class": "job-advantage"}).get_text().replace("\n", "")

    jobdetail = soup.find("div", {"class": "job-detail"}).get_text()
    
    duty_require_list = split_duty_require(jobdetail)
    if len(duty_require_list) != 2:
        duty, require = "", ""
    else:
        duty, require = duty_require_list
        duty = ptxt.Text(duty, 'whi').clean
        require = ptxt.Text(require, 'whi').clean
    
    item['category'] = "_".join(html_file.split("/")[-2].split("_")[:-1])
    item['pname'] = html_file
    item['company'] = company
    item['jobname'] = jobname
    item['salary'] = salary
    item['labels'] = labels
    item['request'] = request
    item['jobadv'] = jobadv
    item['duty'] = "职位职责：" + duty
    item['require'] = "职位要求：" + require
    item['jobdetail'] = jobdetail
    item['dtime'] = dtime
    
    return item

## 模型

### 分词词性

In [48]:
ID = "16302011"
AK = "jDOKyp4qpzeXFHSq91h7K0AG"
SK = "70l8NbS35BG8NuiKhbzEQDBVWdX0HysS"

In [50]:
client = AipNlp(ID, AK, SK)

In [ ]:
# data = pmag.MagicDict()
for file in os.listdir(out_path):
    if file[0] == '.':
        continue
    fname = os.path.join(out_path, file)
    outname = os.path.join(seg_path, file)
    if os.path.exists(outname):
        continue
    data = dict()
    cate, fduty, frequire, fdtimes = get_cate_res(fname)
    data['cate'] = cate
    data['duty'] = fduty
    data['require'] = frequire
    data['dtimes'] = fdtimes
    piop.write_json(outname, data, indent=4, ensure_ascii=False)

In [123]:
def get_cate_res(cate_extract_file):
    cate_item = piop.read_json(cate_extract_file)
    dtimes = []
    require = []
    duty = []
    for item in cate_item:
        dtimes.append(item['dtime'])
        if len(item['require']) < 10:
            continue
        try:
            item_require = segpos(item['require'])
            require.extend(item_require)
        except Exception as e:
            print("GET ITEM ERROR.", e)
            continue
        try:
            item_duty = segpos(item['duty'])
            duty.extend(item_duty)
        except Exception as e:
            print("GET ITEM ERROR.", e)
            continue
    try:
        cate = item['category']
    except Exception as e:
        cate = ""
    return cate, duty, require, dtimes

In [118]:
def segpos(text):
    res = []
    try:
        bd_resp = client.lexer(text)
    except Exception as e:
        print("BaiDu Error:", e)
    for item in bd_resp['items']:
        pos = item['pos']
        w = item['item']
        res.append((w, pos))
    return res

### 过滤

In [6]:
def list2ngrams(lst, n, exact=True):
    """ Convert list into character ngrams. """
    return ["||".join(lst[i:i+n]) for i in range(len(lst)-(n-1))]

In [7]:
def every_in_list(lst1, lst2):
    res = []
    for item in lst1:
        if item in lst2:
            continue
        else:
            res.append(item)
    if len(res) == 0:
        return True
    else:
        return False

In [8]:
def atleast_one_in_list(lst1, lst2):
    res = []
    for item in lst1:
        if item in lst2:
            res.append(item)
    if len(res) == 0:
        return False
    else:
        return True

In [9]:
def get_common(wp_list, ignore, need):
    res = []
    for item in list2ngrams([w+"||"+p for (w,p) in wp_list], 2):
        tmp = item.split("||")
        ww = [tmp[0], tmp[2]]
        if atleast_one_in_list(ww, ignore):
            continue
        pp = [tmp[1], tmp[3]]
        if every_in_list(pp, need):
            res.append(("".join(ww))) #  + " " + " ".join(pp)
    
#     return [(wp, f) for (wp, f) in Counter(res).most_common() if f >= 5]
    return Counter(res).most_common(100)

In [10]:
def filter_require(duty, require):
    res = []
    duty_wlist = [w for w,f in duty]
    for item in require:
        if item[0] in duty_wlist:
            continue
        else:
            res.append(item)
    return res

In [11]:
def get_need_item(wf_list):
    res = pmag.MagicDict()
    n = sum([f for (w,f) in wf_list])
    for w,f in wf_list:
        res[w]['freq'] = f
        res[w]['prob'] = f/n
    return res

In [12]:
def get_dtime_item(dtime_list):
    res = {}
    count = len(dtime_list)
    if count == 0:
        res['count'] = 0
        res['continuous_freq'] = 0
        res['interval_freq'] = 0
        res['publish_freq'] = 0
    else:
        sorted_dt = sorted(dtime_list)
        begin = datetime.datetime.strptime(sorted_dt[0], "%Y-%m-%d")
        end = datetime.datetime.strptime(sorted_dt[-1], "%Y-%m-%d")
        res['count'] = count
        res['continuous_freq'] = count/((end-begin).days+1)
        res['interval_freq'] = count/len(set(dtime_list))
        res['publish_freq'] = len(set(dtime_list))/((end-begin).days+1)
    return res

In [13]:
IGNORE = piop.read_lines('./model/stop_words.txt')
NEEDPOS = ['n', 'nz', 'vn', 'a', 'an']
seg_files = sorted(os.listdir(seg_path))

In [15]:
res = pmag.MagicDict()
for cate, _post in cate_gw.items():
    duty_list, require_list, dtimes_list = [], [], []
    tmp = pmag.MagicDict()
    for post, job in _post.items():
        tmp_duty_list, tmp_require_list, tmp_dtimes_list = [], [], []
        tag = cate+"_"+post
        for file in seg_files:
            if tag in file:
                fname = os.path.join(seg_path, file)
                cate_data = piop.read_json(fname)
                tmp_duty_list.extend(cate_data['duty'])
                tmp_require_list.extend(cate_data['require'])
                tmp_dtimes_list.extend(cate_data['dtimes'])
        
        tmp_duty = get_common(tmp_duty_list, IGNORE, NEEDPOS)
        tmp_require = filter_require(tmp_duty, 
                                     get_common(tmp_require_list, IGNORE, NEEDPOS))
        
        tmp[post]['duty'] = get_need_item(tmp_duty)
        tmp[post]['require'] = get_need_item(tmp_require)
        tmp[post]['demand'] = get_dtime_item(tmp_dtimes_list)
        
        duty_list.extend(tmp_duty_list)
        require_list.extend(tmp_require_list)
        dtimes_list.extend(tmp_dtimes_list)
    
    duty = get_common(duty_list, IGNORE, NEEDPOS)
    require = filter_require(duty, get_common(require_list, IGNORE, NEEDPOS))

    res[cate]['duty'] = get_need_item(duty)
    res[cate]['require'] = get_need_item(require)
    res[cate]['demand'] = get_dtime_item(dtimes_list)
    res[cate]['posts'] = tmp

In [16]:
piop.write_json(os.path.join(model_path, "model.txt"), res, indent=4, ensure_ascii=False)

In [6]:
MODEL = piop.read_json(os.path.join(model_path, "model.txt"))

In [11]:
csfs = []
ilfs = []
for cate, _others in MODEL.items():
    for post, others in _others['posts'].items():
        item = MODEL[cate]['posts'][post]['demand']
        csf = item.get('continuous_freq',0)
        ilf = item.get('interval_freq',0)
        csfs.append(csf)
        ilfs.append(ilf)

In [12]:
len(csfs), len(ilfs)

(39, 39)

In [18]:
math.ceil(sum(csfs)), math.ceil(sum(ilfs))

(1814, 1947)

## 使用

In [80]:
def get_chosen(chosen_wlist, wc_dict):
    chosen = []
    for w in chosen_wlist:
        c = wc_dict[w]
        chosen.append((w, c))
    return chosen

In [81]:
init = "http://localhost:8003/api/generate"
anay = "http://localhost:8003/api/choose"

In [89]:
init_data = requests.post(init).json()
like = dict(init_data['like'])
cando = dict(init_data['cando'])
text = ("喜欢：" + ",".join([*like]) + "\n\n" + "满足：" + ",".join([*cando]))
with open("/Users/HaoShaochun/Desktop/pick.txt", "w") as f:
    f.write(text)

In [83]:
picked = piop.read_file("/Users/HaoShaochun/Desktop/pick.txt")
tmp = picked.split("\n")
plike = tmp[0][4:].split(',')
pcando = tmp[-1][4:].split(',')
chosen_like = get_chosen(plike, like)
chosen_cando = get_chosen(pcando, cando)
anay_res = requests.post(anay, json={"user_chosen_dict": 
                                     {"like": chosen_like, "cando": chosen_cando}}).json()

In [84]:
anay_res

[['技术-后端开发', 100],
 ['设计-视觉设计', 94],
 ['销售-销售', 84],
 ['技术-运维', 74],
 ['技术-硬件开发', 73]]

## 其他

In [1032]:
def get_requirelist(text):
    res = []
    pex= re.compile(r'''
                    ([\w+-]+n)+
                    ''', re.UNICODE | re.VERBOSE)
    cleaned_wt = clean_wt(text)
    for item in "".join(cleaned_wt).split('w'):
        for iitem in pex.finditer(item):
            w = iitem.group()
            res.append(w)
    return res

In [33]:
def get_dutylist(text):
    res = []
    pex = re.compile(r'''
                    [\u4e00-\u9fa5]+a[\u4e00-\u9fa5]+n
                    |
                    [\u4e00-\u9fa5]+n[\u4e00-\u9fa5]+v
                    |
                    [\u4e00-\u9fa5]+v[\u4e00-\u9fa5]+n
                    ''', re.UNICODE | re.VERBOSE)
    wt = segtag(text)
    for item in "".join(wt).split('w'):
        for iitem in pex.finditer(item):
            w = iitem.group()
            w = "".join(ptxt.Text(w, 'chi').extract.mats)
            if w in IGNORE:
                continue
            res.append(w)
    return res